# Infinite pool QBC

### Import packages
**Keras**

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Lambda
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import History, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
import tensorflow.keras.backend as K

**Data Science**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.stats as stats
import sklearn.metrics as metrics
import sklearn
print(sklearn.__version__)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import seaborn as sns
sns.set()

**Other**

In [ ]:
from copy import deepcopy
from math import floor
import time

---
### Configuration regarding dimensionality
Create a dataframe containing the available parameters.

In [ ]:
parameters = pd.DataFrame(columns=[
    'parameter',     # Parameter name
    'minimum',       # Minimum value for sampling
    'maximum',       # Maximum value for sampling
    'default',       # Parameter value if this parameter is fixed
    'is_slepton',    # Boolean: indicates if it is part of the slepton sector (at leading order)
    'is_ew',         # Boolean: indicates if it is part of the electroweak sector (at leading order)
    'is_higgs',      # Boolean: indicates if it is part of the higgs sector (at leading order)
    'is_thirdgen',   # Boolean: indicates if it is part of the third generation sector (at leading order)
    'is_squark',     # Boolean: indicates if it is part of the squark sector (at leading order)
    'is_gluino'      # Boolean: indicates if it is part of the gluino sector (at leading order)
])

def add_row(df, name, minimum, maximum, default, types):
    df = df.append({
        'parameter': name,
        'minimum': minimum, 
        'maximum': maximum,
        'default': default,
        'is_slepton': 'slepton' in types,
        'is_ew': 'ew' in types,
        'is_higgs': 'higgs' in types,
        'is_thirdgen': 'thirdgen' in types,
        'is_squark': 'squark' in types,
        'is_gluino': 'gluino' in types}, ignore_index=True)
    return df

parameters = add_row(parameters, 'M1',      -4000,  4000,     1750,     ['ew'])
parameters = add_row(parameters, 'M2',      -4000,  4000,     1750,     ['ew'])
parameters = add_row(parameters, 'M3',       200,   4000,     1750,     ['gluino'])
parameters = add_row(parameters, 'mL1',      90,    4000,     1750,     ['slepton'])
parameters = add_row(parameters, 'mL3',      90,    4000,     1750,     ['slepton'])
parameters = add_row(parameters, 'mE1',      90,    4000,     1750,     ['slepton'])
parameters = add_row(parameters, 'mE3',      90,    4000,     1750,     ['slepton'])
parameters = add_row(parameters, 'mQ1',      200,   4000,     1750,     ['squark'])
parameters = add_row(parameters, 'mQ3',      100,   4000,     1750,     ['squark', 'thirdgen'])
parameters = add_row(parameters, 'mU1',      200,   4000,     1500,     ['squark'])
parameters = add_row(parameters, 'mU3',      100,   4000,     3000,     ['squark', 'thirdgen'])
parameters = add_row(parameters, 'mD1',      200,   4000,     2000,     ['squark'])
parameters = add_row(parameters, 'mD3',      100,   4000,     2000,     ['squark', 'thirdgen'])
parameters = add_row(parameters, 'At',      -8000,  8000,     3200,     ['thirdgen', 'higgs'])
parameters = add_row(parameters, 'Ab',      -4000,  4000,     2000,     ['thirdgen'])
parameters = add_row(parameters, 'Atau',    -4000,  4000,     2000,     ['slepton'])
parameters = add_row(parameters, 'mu',      -4000,  4000,     200,      ['ew', 'higgs'])
parameters = add_row(parameters, 'MA2',      10000, 16000000, 10000000, ['higgs'])
parameters = add_row(parameters, 'tanbeta',  1,     60,       10,       ['ew', 'higgs'])

Using this dataframe, we can select which sampling parameters to use and which parameters to fix. We select these based on the sectors: by excluding sectors from our sampling we can decrease the dimensionality of the parameter space.

In [ ]:
def get_free_parameters(df, fixed_sectors):
    # Get rows to fix
    fixed_rows = pd.Series([False]*19)
    for f in fixed_sectors:
        fixed_rows = fixed_rows | df['is_'+f]
    # Return Series indicating which variables to sample
    return ~fixed_rows

Let us plot the dimensionality of the sampling space if we increase the number of sectors to remove from the parameter space. The ordering of the sectors is chosen such that the sectors with the weakest ALTAS limits are removed first.

In [ ]:
sectors = ['slepton', 'ew', 'higgs', 'thirdgen', 'squark', 'gluino']
x = np.arange(len(sectors)+1)
dimensionality = []

for i in range(len(sectors)+1):
    fixed_sectors = sectors[:i]
    sample = get_free_parameters(parameters, fixed_sectors)
    dimensionality.append( np.sum(1*sample) )

plt.figure(figsize=(16,4))
plt.scatter(x, dimensionality)
plt.xticks(x, (['-']+sectors))
plt.yticks(np.arange(0, 24, step=4))
for i, dim in enumerate(dimensionality):
    plt.text(x[i]+0.05, dimensionality[i]-0.2, dim)
plt.xlabel('Removed sectors (f.l.t.r. incremental)')
plt.ylabel('Number of dimensions')
plt.show()

In [ ]:
sectors = ['thirdgen']
sample = get_free_parameters(parameters, sectors)
print(parameters[~sample][['parameter', 'default']])

---
### Functions
**Plot Projection**

In [ ]:
def plot_projection(data, classification, x=0, y=1, bins=100):
    # Calculate ranges and bins
    xmin, xmax = np.amin(data[:,x]), np.amax(data[:,x])
    ymin, ymax = np.amin(data[:,y]), np.amax(data[:,y])
    xbins = np.linspace(xmin, xmax, bins)
    ybins = np.linspace(ymin, ymax, bins)
    # Calculate two histograms
    allowed, _, _ = np.histogram2d(data[classification==1.0, x], data[classification==1.0, y], [xbins, ybins])
    excluded, _, _ = np.histogram2d(data[classification==0.0, x], data[classification==0.0, y], [xbins, ybins])
    # Calculate map
    mapping = allowed / (allowed + excluded)
    mapping = mapping.T
    mapping = np.flipud(mapping)
    # Plot
    f, a = plt.subplots(1,1,figsize=(8,8))
    a.matshow(mapping, extent=(xmin, xmax, ymin, ymax), cmap="seismic_r")
    print("({}, {})".format(np.amin(mapping),np.amax(mapping)))
    plt.show()

**Normalisation**

In [ ]:
def normalize(data):
    # Normalize data
    mu = (parameters.maximum.values + parameters.minimum.values)/2
    sigma = (parameters.maximum.values - parameters.minimum.values)/np.sqrt(12)
    data = (data - mu)/sigma
    # Return
    return data

def undo_normalize(data):
    # Normalize data
    mu = (parameters.maximum.values + parameters.minimum.values)/2.0
    sigma = (parameters.maximum.values - parameters.minimum.values)/np.sqrt(12)
    data = data*sigma + mu
    # Return
    return data

**Oracle**

In [ ]:
def oracle(data):
    oracle_model = load_model("susyai.hdf5")
    y = 1.0*(oracle_model.predict(data)[:,1] > 0.5)
    del(oracle_model)
    return y

**Generate data**

In [ ]:
def generate_data(N, parameters, sample_selection, normalized=True):
    # Set fixed parameters
    X = np.ones((N, len(parameters))) * parameters.default.values
    # Set sampling parameters
    X[:,sample_selection] = np.random.rand(N, np.sum(1*sample_selection))
    X[:,sample_selection] *= (parameters[sample_selection].maximum - parameters[sample_selection].minimum).values
    X[:,sample_selection] += parameters[sample_selection].minimum.values
    X_normed = normalize(X)
    # Get labels for points
    y = oracle(X_normed).astype(np.float)
    if normalized:
        return (X_normed,y)
    return (X,y)

**Train Model**

In [ ]:
def train_model(Xtrain, ytrain):
    est = RandomForestClassifier(n_estimators=200, n_jobs=-1)
    est.fit(Xtrain, ytrain)
    return est

**Test Model**

In [ ]:
def test_model(model, Xtest):
    prediction = model.predict_proba(Xtest)[:,1]
    #print(prediction)
    info = 1 - 2*np.abs(prediction-0.5)
    return (prediction, info)

**Show model uncertainty results**

In [ ]:
def uncertainty_results(X, truth, prediction, info_measure, x=0, y=2):
    print("{:<20}{}".format("truth.shape", ytest.shape))
    print("{:<20}{}".format("pred.shape", ypred.shape))
    print("{:<20}{}".format("info.shape", ysigma.shape))

    f, a = plt.subplots(2,2, figsize=(16,16))
    a[0,0].scatter(X[:,x], X[:,y], c=truth.ravel(), cmap="seismic_r")
    a[0,0].set_title("Truth")
    a[0,1].scatter(X[:,x], X[:,y], c=prediction.ravel(), cmap="seismic_r")
    a[0,1].set_title("Prediction")
    a[1,0].scatter(X[:,x], X[:,y], c=np.abs(truth-prediction), cmap="Reds")
    a[1,0].set_title("Difference")
    a[1,1].scatter(X[:,x], X[:,y], c=info_measure, cmap="Purples")
    a[1,1].set_title("Uncertainty")
    plt.show()

---
### Setup experiment

In [ ]:
#free = get_free_parameters(parameters, ['slepton', 'ew', 'higgs', 'thirdgen', 'squark'])
#free = get_free_parameters(parameters, ['slepton', 'ew', 'higgs', 'thirdgen'])
#free = get_free_parameters(parameters, ['slepton', 'ew', 'higgs'])
#free = get_free_parameters(parameters, ['slepton', 'ew'])
#free = get_free_parameters(parameters, ['slepton'])
free = get_free_parameters(parameters, [])
n_free = np.sum(1*free)

In [ ]:
N = 25
x, y = generate_data(N, parameters, free, True)

x = (x*256).astype(np.int)

f, ax = plt.subplots(figsize=(8,8))

ax.matshow(x)

for i,f in enumerate(free):
    if not f:
        p = patches.Rectangle((i-0.5, -.5), 1, N, fill=False, hatch='/////')
        ax.add_patch(p)

plt.xticks(np.arange(19), parameters.parameter.values.tolist(), rotation=90)
plt.grid(False)
plt.show()

---
### Active learning
**Configuration**

In [ ]:
size_start = int(10000*((n_free/19)**2))       # Start size
size_iter = int(2500*((n_free/19)**2))         # Number of data points added in each step
size_sample = int(100000*((n_free/19)**2))     # Size of set to be checked for uncertainty
size_max = int(100000*((n_free/19)**2))        # Maximum size of data set
size_test = 1000000                            # Size of test set
niterations = 7                                # Number of iterations

**Active Sampling function**

In [ ]:
def active_sampling(model, Nquery, Nselect, random_fraction=0.5):
    # Select actively
    select_active = round((1-random_fraction)*Nselect)
    if select_active > 0:
        # Get uncertainty measure
        X, _ = generate_data(Nquery, parameters, free)
        predictions, info = test_model(model, X[:,free])   
        keysort = np.argsort(info)[::-1]
    
        selected = X[keysort[:select_active]]
        method = np.zeros(Nselect)
        method[:select_active] = 1.0
    else:
        selected = None

    # Add random
    select_random = Nselect - select_active
    if select_random > 0:
        X, _ = generate_data(select_random, parameters, free)
        if selected is None:
            selected = X
        else:
            selected = np.vstack((selected, X))

    # Label and return
    prediction = oracle(selected)
    return (selected, prediction)

**Get model performance**

In [ ]:
def model_performance(model, Xtest, ytest):
    ypred = model.predict(Xtest)
    bce = log_loss(ytest, ypred)
    acc = accuracy_score(ytest, 1.0*(ypred>0.5))
    return {"bce":bce, "acc":acc}

**Create logbooks**

In [ ]:
log_AL = open("log_active_learning.csv", "w")
log_AL.write("iteration,size,bce,acc,dt\n")
log_AL.flush()

log_RS = open("log_random_sampling.csv", "w")
log_RS.write("iteration,size,bce,acc,dt\n")
log_RS.flush()

def log_result(log, iteration, size, bce, acc, dt):
    log.write("{},{},{},{},{}\n".format(iteration, size, bce, acc, dt))
    log.flush()

**Run Active Learning**

In [ ]:
if not os.path.exists('arrays'):
    os.mkdir('arrays')

In [ ]:
for iteration in range(niterations):
    print("ITERATION {}".format(iteration))

    # Calculate time to sample all data randomly
    Xtrain_RS, ytrain_RS = generate_data(size_max, parameters, free)
    t_start_rs = time.time()
    Xnew, ynew = generate_data(size_max, parameters, free)
    Xtrain_RS = np.vstack((Xtrain_RS, Xnew))
    ytrain_RS = np.hstack((ytrain_RS, ynew))
    t_end_rs = time.time()
    dt_random = t_end_rs - t_start_rs
    
    # Data set creation
    Xtrain_AL, ytrain_AL = generate_data(size_start, parameters, free)
    Xtrain_RS, ytrain_RS = deepcopy(Xtrain_AL), deepcopy(ytrain_AL)
    Xtest, ytest = generate_data(size_test, parameters, free)
    print("{} of {} points is allowed".format(np.sum(ytest), len(ytest)))

    while len(Xtrain_AL) < size_max:
        K.clear_session()
        
        """ ACTIVE LEARNING """
        t_start_al = time.time()
        # Train model for active learning
        model = train_model(Xtrain_AL[:,free], ytrain_AL)
        # Active sampling of new points
        Xnew, ynew = active_sampling(model, size_sample, size_iter, random_fraction=0.0)
        # Append new points to active learning
        Xtrain_AL = np.vstack((Xtrain_AL, Xnew))
        ytrain_AL = np.hstack((ytrain_AL, ynew))
        t_end_al = time.time()
        # Test model performance
        performance_AL = model_performance(model, Xtest[:,free], ytest)
        # Store performance
        log_result(log_AL, iteration, len(Xtrain_AL)-size_iter, performance_AL["bce"], performance_AL["acc"], t_end_al-t_start_al)
        
        """ RANDOM SAMPLING """
        # Train model for random sampling
        model = train_model(Xtrain_RS[:,free], ytrain_RS)
        # Sample new points
        Xnew, ynew = generate_data(size_iter, parameters, free)
        # Append new points to random sampling
        Xtrain_RS = np.vstack((Xtrain_RS, Xnew))
        ytrain_RS = np.hstack((ytrain_RS, ynew))
        # Test model performance
        performance_RS = model_performance(model, Xtest[:,free], ytest)
        # Store performance
        log_result(log_RS, iteration, len(Xtrain_RS)-size_iter, performance_RS["bce"], performance_RS["acc"], dt_random)

        
        """ LOG AND OUTPUT RESULTS """
        # Screen
        #print(Xtrain_AL.shape, Xtrain_RS.shape)
        print("iteration: {:<5}     size: {:<5}    al-bce: {:<10}    al-acc: {:<10}    rs-bce: {:<10}    rs-acc: {:<10}".format(
            iteration,
            len(Xtrain_AL)-size_iter,
            round(performance_AL["bce"],6),
            round(performance_AL["acc"],6),
            round(performance_RS["bce"],6),
            round(performance_RS["acc"],6)
        ))
    # Output data sets to file
    al = np.hstack((Xtrain_AL, ytrain_AL.reshape(-1,1)))
    rs = np.hstack((Xtrain_RS, ytrain_RS.reshape(-1,1)))
    np.savetxt("arrays/active_{}.csv".format(iteration), al, delimiter=",")
    np.savetxt("arrays/random_{}.csv".format(iteration), rs, delimiter=",")